In [1]:
import pandas as pd
import getpass
import sqlalchemy as sa
import numpy as np
import seaborn as sns
import matplotlib as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder  ##. better to use dummy from pandas 
from sklearn.preprocessing import PowerTransformer
from scipy.stats import boxcox
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.feature_selection import RFE
pd.options.display.max_rows = 50
## Install xlrd package to load Excel files
# conda install openpyxl
## conda install xlrd

C:\Users\Lenovo\anaconda3\envs\DA_Env\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


## Importing Dataset.

In [3]:
hp_df_org = pd.read_csv('Data/house_price_df.csv')
hp_df = hp_df_org

## EDA

In [ ]:
hp_df.info()

The Dataset consists of 21597 Houses containing 21 features. Price is to be the predicted variables. 

In [ ]:
hp_df.describe()

The average house has about 3 bedrooms, 2080 sqft of living area, a lot size of 15000 sqft, was built in 1970 and has a price of 540000 $. The mean and the median differ alot in the variable sqft lot, where the median is much lower then the mean. This leads to the conculusion that many houses don't have a big lot but a few house with a big lot change the distribution.

In [ ]:
# sns.pairplot(hp_df) #To view remove comment. Commented to get higher performance

The Pairplot gives first interesting insides. Obviously there are linear tendencies between variables of size like lot size, size of living room. Many variables are categorical or boolean and should be transformed.

## Taking a closer look at certain variables

### Date

In [ ]:
print(hp_df['date'].min(), hp_df['date'].max())

The date variable displays the date when the house was sold. Data is between May 2014 and May 2015. 

In [ ]:
hp_df['bathrooms'].value_counts()

Most houses have 2,5 bathrooms, a lot also only . A view outliers have a very high number of bathrooms.

## Location

For the location the datasets offers three features. Latitute and Longitude (only useful combined) and zipcodes. Let's look at both of them.

In [ ]:
 sns.scatterplot(data=hp_df, x='long', y='lat', hue = 'waterfront').set(title='Houses at waterfront')

Looking at the geographical location of the houses and displaying the lot size it is visual, that the properties at the are outskirts and the countryside get bigger. 

In [ ]:
 sns.scatterplot(data=hp_df, x='long', y='lat', hue = 'zipcode').set(title='Zipcode and geographical location')

The zipcodes are not in a logical order so the variables should not be used as a numerical variables.
And alternative idea would be to use the travel distance from the house to the city center.

In [ ]:
distance = pd.read_csv('Data/distance.csv')

In [ ]:
hp_df['distanceM'] = distance['DistanceM'].astype('float64')

In [ ]:
sns.scatterplot(data=hp_df, x='long', y='lat', hue = 'distanceM').set(title='Geographical location and distance to citycenter by car')

## Year renovated

In [ ]:
sns.histplot(data=hp_df, x='yr_renovated').set(title= 'Histogram of Year renovated')

The year renovated displays a 0 if the house was not yet renovated. This is not useful information in a linear regression. 

Workaround:

introduce boolean variable: Renovated: Yes/N0
Put zero values as No  


In [ ]:
hp_df['renovated'] = bool

In [ ]:
hp_df.loc[hp_df['yr_renovated'] == 0, 'renovated'] = False

In [ ]:
hp_df.loc[hp_df['yr_renovated'] != 0, 'renovated'] = True

## Basement

In [ ]:
sns.histplot(data=hp_df, x='sqft_basement')

Most Houses don't have a basement, so a dummy will be introduced checking for basement or not:

In [ ]:
hp_df.loc[hp_df['sqft_basement'] == 0, 'basement'] = False
hp_df.loc[hp_df['sqft_basement'] != 0, 'basement'] = True

## Grade

Bin grade to have a better classification:

In [ ]:
grade_mean = hp_df['grade'].mean()

In [ ]:
for i in range(0,len(hp_df)):
    if hp_df['grade'][i]<grade_mean:
        hp_df['grade'][i] = 'below_avg'
    else:
        hp_df['grade'][i] = 'avg_or_better'
        

In [ ]:
sns.histplot(hp_df['grade'])

## Floors

In [ ]:
sns.histplot(hp_df['floors'])

In [ ]:
hp_df['floors'].value_counts()

Binning floors to 2 categories: One floor and more than one floor.

In [ ]:
for i in range(0, len(hp_df)):
    if hp_df.loc[i, 'floors'] <= 1.0:
        hp_df.loc[i,'floors'] = 'one'
    else: 
        hp_df.loc[i, 'floors'] = 'more_than_one'

In [ ]:
sns.histplot(hp_df['floors'])

## Dropping columns:

For now the columns Unnamed, id, date and long, lat will be dropped from the dataframe.

In [ ]:
col_drop = ['Unnamed: 0',\
            'id',\
            'date',\
           'long',\
            'sqft_basement',\
            'yr_renovated',\
           'lat']   
# 'Unnamed: 0' : id from import without information
# 'id': random or consequtive values without values
# 'date': probably the date the house was added to the database, no additional information for houseprice
# 'lang', 'lat': Geografical data, not usable in linear regression like this, can be converted to zones using knn. 
#    Task for later. But zip codes are enough, probably


In [ ]:
hp_df = hp_df.drop(col_drop, axis= 1)

## Changing datatypes


The format of the features are mostly numerical (int or float). Many of the variables have to be transformed to category to give meaningful results in a regression.
These are:

'zipcode, 'bedrooms', 'bathrooms', 'floors', 'waterfront', 'view', 'condition', 'grade'

In [ ]:
hp_df.info()

In [ ]:
# From the pairplot we can see which columns are categorical/dummies:
#  bderooms, bathrooms, floors, waterfront, view, condition, grade
ordinal_var = ['zipcode', 'floors', 'waterfront', 'view', 'condition', 'grade']
hp_df[ordinal_var] = hp_df[ordinal_var].astype('category')
hp_df.info()

In [ ]:
hp_df.shape

## Checking correlation of numerical variables

In [ ]:
corr = hp_df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(9, 7))
    ax = sns.heatmap(corr, mask=mask,cmap='coolwarm', vmin=-1,vmax=1,annot=True, square=True)

The features should not be correlated in order to fulfill the asumtions of linear regression.
As expected there is a lot of correlation between sqf_above, sqf_living and other measures of size. 
- First: Drop sqf_above

In [ ]:
corr_matrix=hp_df.corr().abs()
upper_triangle=corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(bool))
corr_var = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.80)]
hp_df = hp_df.drop(corr_var, axis= 1)
hp_df.shape

In [ ]:
corr = hp_df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(9, 7))
    ax = sns.heatmap(corr, mask=mask,cmap='coolwarm', vmin=-1,vmax=1,annot=True, square=True)

The new dataframe does not contain anymore variables with a higher correlation than 0.8, but some variable still have high correlation which could be analysed later.

# Normalization

## Histograms
To check for the assumtion of normal distributions we check the histograms of the numerical variables

In [ ]:
hp_df.select_dtypes(['int64','float']).hist(figsize=(12,12), bins=40)


All numerical varibales are non-normaly distributed and need to be transformed. 

## Transforming to normal distribution

In [ ]:
hp_df.select_dtypes(['int64','float']).columns

In [ ]:
# List of variables names to be transformed: 
to_trans =  {'sqft_living': 'qt',\
             'sqft_lot': 'qt',\
             'yr_built': 'qt',\
             'sqft_living15': 'qt',\
             'sqft_lot15': 'qt',\
            'bedrooms': 'qt',\
            'bathrooms': 'qt',\
            'distanceM': 'qt'}

For the transformation of the variables the Quantile Transformer is applied. 

In [ ]:

pt = PowerTransformer()
qt = QuantileTransformer(output_distribution="normal")

transformed_cols = []
for i in to_trans.keys():
    if list(to_trans.values())[0] == 'pt':
        transformed = pt.fit_transform(hp_df[i].to_numpy().reshape(-1,1))
    elif list(to_trans.values())[0] == 'qt':
        transformed = qt.fit_transform(hp_df[i].to_numpy().reshape(-1,1))
    else: 
        print('no transformer could be identified')
    col_name = i+'_transformed'
    #sns.displot(transformed).set(title=f'Histogram of {col_name}')
    hp_df[col_name] = transformed
    transformed_cols = transformed_cols + [col_name]
    hp_df = hp_df.drop(i, axis= 1)
    
    
hp_df.describe()

In [ ]:
transformed_cols

In [ ]:
hp_df.select_dtypes(['int', 'float64']).hist(figsize=(12,12), bins=40)

## Remove outliers

The Datasets contains a few datapoints that can described as outliers. One house for example as 33 rooms while the average amount of rooms is around 3. Using the two time the Interquartile range whe remove the outliers. 

In [ ]:
for i in transformed_cols:
    sns.boxplot(data=hp_df[transformed_cols]).set(title='Boxplots of numerical variables showing outliers')
    plt.xticks(rotation=90)

In [ ]:
 len_before = hp_df.shape[0]

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-2*iqr
    fence_high = q3+2*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

for i in transformed_cols: 
    hp_df = remove_outlier(hp_df, i )

In [ ]:
for i in transformed_cols:
    sns.boxplot(data=hp_df[transformed_cols]).set(title='Boxplots of numerical variables without outliers')
    plt.xticks(rotation=90)

In [ ]:
len_after = hp_df.shape[0]

In [ ]:
print(f'Of the original dataset {round(len_after/len_before * 100, 2)}% of rows remain')

## One-hot encode categorical variables

In [ ]:
col_names_object = list(hp_df.select_dtypes("object").columns)
col_names_object.extend(hp_df.select_dtypes("category").columns)
col_names_object

In [ ]:
cat_features =  pd.get_dummies(hp_df[col_names_object] , drop_first=True)

In [ ]:
hp_df = hp_df.drop(col_names_object, axis = 1)

In [ ]:
hp_df[cat_features.columns] = cat_features

In [ ]:
list(hp_df.columns)

## Linear Regression

We're splitting into X and y (predictors and target variable).


In [ ]:
def Xy_split(df, output_col):
    """ Split the dataset into:
        X = dataframe of input variables
        y = predicted variable
    """
    X=df.drop(output_col, axis=1)
    y=df[output_col]
    return X, y

X,y = Xy_split(hp_df, 'price') 

We're creating train and test-sets.

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=.30,random_state=100)

We're scaling the data using Standard Scaler.

In [ ]:
std_scaler=StandardScaler().fit(X_train.select_dtypes(['int', 'float64']))
X_train_scaled=std_scaler.transform(X_train.select_dtypes(['int', 'float64']))
X_test_scaled=std_scaler.transform(X_test.select_dtypes(['int', 'float64']))

In [ ]:
X_train.describe()

We're concatenating the data scaled numerical data with the categorical variables.

In [ ]:
X_train_scaled = np.concatenate((X_train_scaled, np.asarray(X_train[list(cat_features.columns)])), axis = 1) 

X_test_scaled = np.concatenate((X_test_scaled, np.asarray(X_test[list(cat_features.columns)])), axis =1)
print(X_train_scaled.shape, X_test_scaled.shape)

We're modeling using linear regression and desplaying key indicators to interpret the model fit.

In [ ]:
model=LinearRegression()    # model
model.fit(X_train_scaled, y_train) 


y_pred=model.predict(X_test_scaled)   # model prediction
y_pred_train=model.predict(X_train_scaled)


result=pd.DataFrame({"y_test":y_test,"y_pred":y_pred})

fig, ax = plt.subplots(1,3,figsize=(14,4))
ax[0].plot(y_pred, y_test, 'o')
ax[0].set_xlabel("y_test")
ax[0].set_ylabel("y_pred")
ax[0].set_title("Test Set -Predicted vs real")

ax[1].hist(y_test - y_pred)
ax[1].set_xlabel("Test y-y_pred")
ax[1].set_title("Test Set Residual histogram")

ax[2].plot(y_pred,y_test - y_pred,"o")
ax[2].set_xlabel("predicted")
ax[2].set_ylabel("residuals")
ax[2].set_title("Residuals by Predicted")
ax[2].plot(y_pred,np.zeros(len(y_pred)),linestyle='dashed')

The scatterplot of the target variable and the predicted variables displays a obvious linear trend. Risiduals seem to be normaly distributed. The Residual are not perfectly randomly distributed around 0, but overall the model seems to be okay. 

In [ ]:
print('MSE test: ',  round(mse(y_test,y_pred),2))
print('MSE train: ', round(mse(y_train,y_pred_train),2))
print('RMSE test: ', round(np.sqrt(mae(y_test,y_pred)),2))
print('MSE train - MSE test: ', round(mse(y_train,y_pred_train) - mse(y_test,y_pred),2) )

print('MAE test: ', round(mae(y_test,y_pred),2))

R2_test=model.score(X_test_scaled,y_test)
R2_train=model.score(X_train_scaled,y_train)
print('R2_test: ', round(R2_test, 2))
print('R2_train: ', round(R2_train, 2))

Adj_R2= 1 - (1-R2_test)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print('Adj_R2:', round(Adj_R2,2))

In [ ]:
features_importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': abs(model.coef_)
})
features_importances = features_importances.sort_values(by='Importance', ascending=False)
features_importances
plt.figure(figsize=(21,14))
plt.bar(x=features_importances['Attribute'], height=features_importances['Importance'], color='#087E8B')
plt.title('Feature importances obtained from coefficients', size=20)
plt.xticks(rotation='vertical')
plt.figure(figsize=(20,50))
plt.show()

In [ ]:
features_importances.nlargest(10, 'Importance')

The Top 10 most important features are mostly zipcodes. Also waterfront and view play an important role.

## Get P-values from model 

To filter the significant columns we get the p-value from the linear regression. If it is higher than 0.5 the column is dropped.

In [ ]:
X_train_const_scaled = sm.add_constant(X_train_scaled) #getting model using sm.Ols (more detailed statistics)
model = sm.OLS(y_train, X_train_const_scaled).fit()

In [ ]:
sign_cols = model.pvalues[1:len(model.pvalues)]  # remove constant
sign_cols = pd.DataFrame([sign_cols])


In [ ]:
X_col_list = X.columns
X_dropped_p = X
for i in range(0,len(X.columns)):
    if sign_cols.iloc[0,i]>0.05:
        X_dropped_p = X_dropped_p.drop(X_col_list[i], axis=1)

In [ ]:
print(f'From orginally {X.shape[1]} features {X_dropped_p.shape[1]} features remain.')

## 3d linear Model only including variabels that had a siginificant p-value in the 2s Model

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X_dropped_p, y, test_size=.30,random_state=100)

In [ ]:
X_train_num = X_train.select_dtypes(['float64'])
X_test_num = X_test.select_dtypes(['float64'])
X_train_cat = X_train.select_dtypes(['uint8'])
X_test_cat = X_test.select_dtypes(['uint8'])
print(X_train_num.shape, X_test_num.shape, X_train_cat.shape, X_test_cat.shape)

In [ ]:
std_scaler=StandardScaler().fit(X_train_num)   ##. finding the parameters ( mean, variance from the training set )
                                            ## 
X_train_scaled=std_scaler.transform(X_train_num)

X_test_scaled=std_scaler.transform(X_test_num)
X_test_scaled.shape

In [ ]:
X_train_scaled = np.concatenate((X_train_scaled, np.asarray(X_train_cat)), axis = 1) 

X_test_scaled = np.concatenate((X_test_scaled, np.asarray(X_test_cat)), axis =1)
print(X_train_scaled.shape, X_test_scaled.shape)

In [ ]:
model=LinearRegression()    # model
model.fit(X_train_scaled, y_train) 


y_pred=model.predict(X_test_scaled)   # model prediction
y_pred_train=model.predict(X_train_scaled)


result=pd.DataFrame({"y_test":y_test,"y_pred":y_pred})

fig, ax = plt.subplots(1,3,figsize=(14,4))
ax[0].plot(y_pred, y_test, 'o')
ax[0].set_xlabel("y_test")
ax[0].set_ylabel("y_pred")
ax[0].set_title("Test Set -Predicted vs real")

# Get a histogram of the residuals ie: y - y_pred.  Homoscdasticity
# It resembles a normal distribution?
ax[1].hist(y_test - y_pred)
ax[1].set_xlabel("Test y-y_pred")
ax[1].set_title("Test Set Residual histogram")

ax[2].plot(y_pred,y_test - y_pred,"o")
ax[2].set_xlabel("predicted")
ax[2].set_ylabel("residuals")
ax[2].set_title("Residuals by Predicted")
ax[2].plot(y_pred,np.zeros(len(y_pred)),linestyle='dashed')

In [ ]:
print('MSE test: ',  round(mse(y_test,y_pred),2))
print('MSE train: ', round(mse(y_train,y_pred_train),2))
print('RMSE test: ', round(np.sqrt(mae(y_test,y_pred)),2))
print('MSE train - MSE test: ', round(mse(y_train,y_pred_train) - mse(y_test,y_pred),2) )

print('MAE test: ', round(mae(y_test,y_pred),2))

R2_test=model.score(X_test_scaled,y_test)
R2_train=model.score(X_train_scaled,y_train)
print('R2_test: ', round(R2_test, 2))
print('R2_train: ', round(R2_train, 2))

Adj_R2= 1 - (1-R2_test)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print('Adj_R2:', round(Adj_R2,2))

In [ ]:
features_importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': abs(model.coef_)
})
features_importances = features_importances.sort_values(by='Importance', ascending=False)
features_importances
plt.figure(figsize=(20, 15))
plt.bar(x=features_importances['Attribute'], height=features_importances['Importance'], color='#087E8B')
plt.title('Feature importances obtained from coefficients', size=20)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
features_importances.nlargest(10, 'Importance')

In [ ]:
#features_importances.to_csv('Data/final_model_feature_importance.csv')

In [ ]:
X_train_const_scaled = sm.add_constant(X_train_scaled) # adding a constant

model = sm.OLS(y_train, X_train_const_scaled).fit()
predictions_train = model.predict(X_train_const_scaled) 

X_test_const_scaled = sm.add_constant(X_test_scaled) # adding a constant
predictions_test = model.predict(X_test_const_scaled) 
print_model = model.summary()
print(print_model)

## Visualizing results:

Zip-code showed to be very determining for prices. The zipcode 98039 showed to show the highest price effect.

In [ ]:
features_importances

In [ ]:
plt.hist(features_importances['Importance'])

In [ ]:
features_importances['zip_rank'] = pd.qcut(features_importances['Importance'], q=4, labels=["low", "medium", "high", "premium"])

rank into Quartiles

Get only zipcodes to compare:

In [ ]:
importance_zipcode = features_importances[features_importances['Attribute'].str.startswith('zipcode')]

In [ ]:
importance_zipcode['zipcode']=importance_zipcode['Attribute'].str.replace('zipcode_', '')

In [ ]:
importance_zipcode['zipcode'] =importance_zipcode['zipcode'].astype('int')

In [ ]:
hp_df_org.info()

In [ ]:
df_result = pd.merge(importance_zipcode[['zipcode', 'zip_rank']], hp_df_org, on='zipcode' )

In [ ]:
df_result['zip_rank'].sort_values()

In [ ]:
plt.figure(figsize=(13,8))
sns.scatterplot(data=df_result, x='long', y='lat', hue='zip_rank').set(title= 'Zipcodes ordered by importance')

Run model again with rated zipcodes to get more precise results.

In [ ]:
hp_df_org['price_rank'] = pd.qcut(hp_df_org['price'], q=4, labels=["low", "medium", "high", "premium"])

In [ ]:
sns.scatterplot(data=hp_df_org, x='long', y='lat', hue='price_rank').set(title= 'Prices ordered geographically')

In [ ]:
avg_price_zip = hp_df_org[['zipcode', 'price']].groupby('zipcode').mean('price')
avg_price_zip['avg_price_zip']=avg_price_zip['price']
avg_price_zip.drop('price', axis=1, inplace=True)
avg_price_zip

In [ ]:
merged_av_price = pd.merge(avg_price_zip[['avg_price_zip']], hp_df_org, on='zipcode')
merged_av_price.info()

In [ ]:
sns.scatterplot(merged_av_price, x='long', y='lat', hue='avg_price_zip')